In [59]:
import ase
import numpy as np
import ase.build
from ase.visualize import view
import ase.data.pubchem
from lammpslib import LAMMPSlib
import matplotlib.pyplot as plt
import sys

In [135]:
at = ase.io.read("benzene_test/no_skew_benzene_MD.snapshot.all.extxyz")
at=at[:12]

#at.wrap()


In [136]:
#view(at)

In [137]:
#wrap molecules such that they do cross boundaries
molsize=6
nmols = len(at)//molsize
new_pos=[]
for imol in range(nmols):
    mol=at[imol*molsize:imol*molsize+molsize]
    new_pos.append(mol.get_distances(0,range(len(mol)),True,True)+mol[0].position)
    #at[imol*molsize:imol*molsize+molsize].set_positions(new_pos)
new_pos =np.array(new_pos)
new_pos = np.reshape(new_pos,[-1,3])
at.set_positions(new_pos)

In [138]:
def get_mic(pos,at):
    return pos - np.floor(pos @ np.linalg.inv(at.cell)) @ at.cell

In [139]:
get_mic((104,3,1),at)

array([27.57707268,  3.        ,  1.        ])

In [140]:
def get_com_mic(at):
    """Obtain centre of mass of a molecule, accounting for PBCs.
    returns: centre of mass"""
    dists = at.get_distances(0,range(len(at)),True,True)
    masses = at.get_masses()
    com = (masses @ dists / masses.sum())+at.positions[0]
    return get_mic(com,at) #com reflected back into the cell
    
    

In [141]:
def unwrap_pos(at, mol_size=1):
    "Unwrap molecular coordinates such that molecules straddling pbcs are calculated correctly."
    "Still assuming a single species system"
    mols = len(at)//mol_size
    for i in range(mols):
        dists = at.get_distances(i*mol_size,range(i*mol_size,i*mol_size+mol_size),True,True)
        at.positions[i*mol_size:i*mol_size+mol_size] = dists+at.positions[i*mol_size]
    

In [142]:
def get_moments_of_inertia_mic(at, vectors=False):
    """Get the moments of inertia along the principal axes.

    The three principal moments of inertia are computed from the
    eigenvalues of the symmetric inertial tensor. Periodic boundary
    conditions are NOT ignored.
    """
    
    at.positions = at.get_distances(0,range(len(at)),True,True)+at[0].position
    com = at.get_center_of_mass()
    positions = at.get_positions().copy()
    positions -= com  # translate center of mass to origin
    masses = at.get_masses()

    # Initialize elements of the inertial tensor
    I11 = I22 = I33 = I12 = I13 = I23 = 0.0
    for i in range(len(at)):
        x, y, z = positions[i]
        m = masses[i]

        I11 += m * (y ** 2 + z ** 2)
        I22 += m * (x ** 2 + z ** 2)
        I33 += m * (x ** 2 + y ** 2)
        I12 += -m * x * y
        I13 += -m * x * z
        I23 += -m * y * z

    Itensor = np.array([[I11, I12, I13],
                        [I12, I22, I23],
                        [I13, I23, I33]])
    

    evals, evecs = np.linalg.eigh(Itensor)
    if vectors:
        return evals, evecs.transpose()
    else:
        return evals

In [143]:
#evals,evecs = get_moments_of_inertia_mic(at,True)

In [144]:
def map_velocities(disps, velo):
    velos=np.zeros_like(disps)
    for i in range(len(velo[0])):
        
        disps_del = np.delete(disps,i,axis=1)
        print(velo[0,i])
        velos[:,i] = np.linalg.norm(disps_del,axis=1)*velo[0,i]
        
    return velos

In [198]:
def gen_random_velo_mole(at,KEmax,mol_size):#note need to remove rot velocity weights for molsize 1
    #note change rng when implementing
    nmols = len(at)//mol_size
    nDOF=6
    unit_rv = np.random.normal(0.0,1.0, (2*nmols, 3)) #avoid rng generation twice 
    unit_rv /= np.linalg.norm(unit_rv)
    rv_mag = np.random.uniform()**(1.0/(nDOF*nmols))
    rv_mag = 1.0
    at_velocities=np.zeros([len(at),3])
    for i in range(nmols):
        mass = at[i*mol_size:i*mol_size+mol_size].get_masses().sum()
        trans_velocity = rv_mag * np.sqrt(2.0/np.array([mass,]*3).transpose()) * np.sqrt(KEmax) * unit_rv[i]
        I,evecs = get_moments_of_inertia_mic(at[i*mol_size:i*mol_size+mol_size],True)
        
        print(0.5*np.sum(mass*trans_velocity**2))
        
        rot_velocity  = rv_mag * np.sqrt(2.0/I.T) * np.sqrt(KEmax) * unit_rv[i+nmols]
        print(np.sum(I*rot_velocity**2*0.5))
        dist_vecs = at.positions[i*mol_size:i*mol_size+mol_size] - at[i*mol_size:i*mol_size+mol_size].get_center_of_mass()
        new_basis_vecs = np.dot(evecs,dist_vecs.T)        
        rot_component = np.cross(rot_velocity,new_basis_vecs,axisa=0,axisb=0)
        #print(rot_component)
        at_velocities[i*mol_size:i*mol_size+mol_size] = np.dot(rot_component,evecs)+trans_velocity
        #at_velocities[i*mol_size:i*mol_size+mol_size] = trans_velocity
        
    return at_velocities
        

In [199]:
# def set_image_flags(at):

#     fractional_positions = at.get_scaled_positions(wrap=False)
#     # Calculate the image flags for each atom
#     image_flags = np.floor(fractional_positions)
#     # Print the image flags for each atom
#     for atom_index,image_flag in enumerate(image_flags):
#         if image_flag.any():
#             at.calc.lmp.command(f"set atom {int(atom_index+1)} image {int(image_flag[0])} {int(image_flag[1])} {int(image_flag[2])}")

In [200]:
velo= gen_random_velo_mole(at,5,6)
at.set_velocities(velo)
print(at.get_kinetic_energy())


0.30435672497448885
0.7472430431562788
0.6550992660135084
3.2933009658557246
4.999999999999983


In [149]:
random_ke = []
my_ke = []

for i in range(1000):
    ns_run.py.

SyntaxError: invalid syntax (<ipython-input-149-5a0de39b4165>, line 5)

In [ ]:
lmp_header=["units metal",
            "atom_style angle",
            "atom_modify map array sort 0 0"]

In [ ]:
init_cmds=[
    "pair_style lj/cut 14.00",
"pair_coeff * * 0.00345 1.8",
"pair_modify shift yes",
 "bond_style zero",
 "bond_coeff *",
 "angle_style zero",
 "angle_coeff * 120"
]

In [ ]:
pot = LAMMPSlib(lmpcmds=init_cmds,
          atom_types={"C":1},
          keep_alive=True,
          lammps_name="serial",
          lammps_header=lmp_header,
          read_molecular_info=True,
          log_file="test.log2")

In [ ]:
at.calc=pot
at.calc.start_lammps()
at.calc.parse_bonds(at)

In [ ]:
# lmp.lammpsbc(True)

# lmp.start_lammps()
at.calc.initialise_lammps(at)

#lmp.set_cell(at)
# lmp.set_lammps_pos(at)
at.calc.lmp.command("neigh_modify exclude molecule/intra all")
at.calc.lmp.command("comm_modify vel yes")

at.calc.lmp.command("timestep 0.0001")


at.calc.lmp.command("fix 1 all rigid/nve molecule reinit yes")
#at.calc.lmp.command("fix 1 all nve")

at.calc.lmp.command("dump myDump all atom 1 dump.lammpstrj")
ase.io.write("benzene.data",at,format="lammps-data")

In [ ]:
#at.calc.lmp.command("compute image all property/atom ix iy iz")
#at.calc.lmp.command("compute mol_id all property/atom mol")
at.calc.lmp.command("info all out log")


In [ ]:
#l=at.calc.lmp.numpy.extract_compute("mol_id",1,0)
# at.calc.lmp.command("dump myDump2 all custom 1 moldump.lmp id x y z xu yu zu ix iy iz")
at.calc.lmp.command("thermo 1")
# at.calc.lmp.command("thermo_style custom step temp cella cellb cellc cellalpha cellbeta cellgamma")



In [ ]:


at.calc.propagate(at,["energy","forces"],["positions","numbers","pbc"],0)

for i in range(10):
    at.calc.propagate(at,["energy","forces"],["positions","numbers","pbc"],1000)
    #velo= gen_random_velo_mole(at,5,6)
    at.set_velocities(velo)

    #at.calc.lmp.command("reset_atoms image all")
    print(at.get_kinetic_energy(), at.get_potential_energy())

    


    

In [ ]:
traj=ase.io.read("dump.lammpstrj",index=":")

In [ ]:
view(traj)

In [ ]:
# for i in range(6):
#     plt.plot([j.get_distance(1,i,mic=True,vector=False) for j in traj], label = i)
#     #plt.plot([j.positions[5] for j in traj])
# plt.legend()
# plt.show()

In [ ]:
plt.plot(range(len(velo[::6])), velo[::6])

In [ ]:
range(len)